# How to create a custom tool

When constructing an agent, you will need to provide it with a list of Tools that it can use. Besides the actual function that is called, the Tool consists of several components:

| Attribute       | Type                      | Description                                                                                                      |
|-----------------|---------------------------|------------------------------------------------------------------------------------------------------------------|
| name          | str                     | Must be unique within a set of tools provided to an agent                                           |
| description   | str                     | Describes what the tool does. Used as context by the LLM or agent.                                       |
| args_schema   | Pydantic BaseModel      | Optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters |

LangChain provides a few ways to define tools:

1. @tool decorator -- the simplest way to define a custom tool.
2. Subclass from `BaseTool` -- provides more control and allows providing both sync and async implementations.
3. Using `StructuredTool.from_function` class method, also allows providing both sync and async implementations.

## @tool decorator

This `@tool` decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description - so a docstring MUST be provided. 

In [213]:
from langchain.tools import tool


@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"


# Let's inspect some of the attributes associated with the tool.
print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


To create a tool with multiple inputs you can do the following:

In [214]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(a: int, b: int) -> int - Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


You can also customize the tool name and JSON args by passing them into the tool decorator.

In [215]:
from langchain.pydantic_v1 import BaseModel, Field


class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"


# Let's inspect some of the attributes associated with the tool.

print(search.name)
print(search.description)
print(search.args)
print(search.return_direct)

search-tool
search-tool(query: str) -> str - Look up things online.
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}
True


:::{.callout-tip}

Because tools implement the [Runnable interface](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html), you
can use all the standard Runnable methods with tools!

These methods can be helpful to create test that the tool is implemented correctly.
:::

In [223]:
print(search.invoke({"query": "hello"}))
print(multiply.invoke({"a": 4, "b": 2}))
# And the async variants. These incur a small overhead since they're
# delegating to the sync implementation from another thread.
print(await search.ainvoke({"query": "hello"}))
print(await multiply.ainvoke({"a": 4, "b": 2}))

LangChain
8
LangChain
8


### async

:::{.callout-important}

If you're working in an async codebase, you should create async tools rather than sync tools using the @tool decorator.

Using `ainvoke` using a sync provided tool incurs a small overhead if the tool is not associated with a native
`async` implementation.
:::

:::{.callout-tip}

If you need to have both synd and async implementation for your tool, consider subclassing from `BaseTool` instead.
:::

In [224]:
@tool
async def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


await multiply.ainvoke({"a": 2, "b": 3})

6

:::{.callout-important}

If you've created an async tool. You **cannot** and **should not** use
it via the the sync `invoke`, `batch` or `stream` methods. 

For example, this using `invoke` instead of `ainvoke` will fail by design: `multiply.invoke({'a': 3, 'b': 5})`
:::

## Subclass BaseTool

You can also explicitly define a custom tool by subclassing the BaseTool class. This provides maximal control over the tool definition at the expense of a bit more effort.

In [230]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SearchInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return "LangChain"

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        # Provide a native async implementation here.
        # For example, if you're connection to an API
        # you can use httpx with an async rather than sync client.
        raise NotImplementedError("custom_search does not support async")


class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        # If the calculation is cheap, you can just delegate to the sync implementation
        # as shown below.
        # If teh sync calculation is expensive, you should delete the entire _arun method.
        # LangChain will automatically provide a better implementation that will
        # kick off the task in a thread to make sure it doesn't block other async code.
        return self._run(a, b, run_manager=run_manager.get_sync())

In [231]:
search = CustomSearchTool()
print(search.name)
print(search.description)
print(search.args)

custom_search
useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}


In [239]:
print(search.invoke({"query": "hello"}))
# Or you can use the short hand if there's only a single parameter
print(search.invoke("hello"))

LangChain
LangChain


In [244]:
multiply = CustomCalculatorTool()
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

print(multiply.invoke({"a": 2, "b": 3}))
print(await multiply.ainvoke({"a": 2, "b": 3}))

Calculator
useful for when you need to answer questions about math
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}
True
6
6


## StructuredTool dataclass

You can also use a `StructuredTool` dataclass. This methods is a mix between the previous two. It's more convenient than inheriting from the BaseTool class, but provides more functionality than just using a decorator.



In [245]:
def search_function(query: str):
    return "LangChain"


search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
    # coroutine= ... <- you can specify an async method if desired as well
)

In [246]:
print(search.name)
print(search.description)
print(search.args)

Search
Search(query: str) - useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'type': 'string'}}


You can also define a custom `args_schema` to provide more information about inputs.

In [247]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

In [248]:
print(calculator.name)
print(calculator.description)
print(calculator.args)

Calculator
Calculator(a: int, b: int) -> int - multiply numbers
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}


## Handling Tool Errors 
When a tool encounters an error and the exception is not caught, the agent will stop executing. If you want the agent to continue execution, you can raise a `ToolException` and set `handle_tool_error` accordingly. 

When `ToolException` is thrown, the agent will not stop working, but will handle the exception according to the `handle_tool_error` variable of the tool, and the processing result will be returned to the agent as observation, and printed in red.

You can set `handle_tool_error` to `True`, set it a unified string value, or set it as a function. If it's set as a function, the function should take a `ToolException` as a parameter and return a `str` value.

Please note that only raising a `ToolException` won't be effective. You need to first set the `handle_tool_error` of the tool because its default value is `False`.

In [249]:
from langchain_core.tools import ToolException


def search_tool1(s: str):
    raise ToolException("The search tool1 is not available.")

First, let's see what happens if we don't set `handle_tool_error` - it will error.

In [250]:
search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="A bad tool",
)

search.run("test")

ToolException: The search tool1 is not available.

Now, let's set `handle_tool_error` to be True

In [251]:
search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="A bad tool",
    handle_tool_error=True,
)

search.run("test")

'The search tool1 is not available.'

We can also define a custom way to handle the tool error

In [254]:
def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution: "
        + error.args[0]
        + " Please try another tool."
    )


search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="A bad tool",
    handle_tool_error=_handle_error,
)

search.run("test")

'The following errors occurred during tool execution: The search tool1 is not available. Please try another tool.'